In [ ]:


import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime as dt

from sklearn.experimental import enable_iterative_imputer


from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBRegressor

from sklearn.preprocessing import OrdinalEncoder
os.getcwd()


In [424]:
 # import data sets

soda_data =  pd.read_csv(str(os.getcwd()) + "/soft_drink_sales.csv")
#lower the case of the columns 
soda_data.columns = soda_data.columns.str.lower()

benchmark_data = pd.read_csv(str(os.getcwd()) + "/benchmarks.csv")


In [425]:
#no missing values

benchmark_data.info()
benchmark_data.isnull().sum() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   yearmon    12 non-null     object 
 1   benchmark  12 non-null     float64
dtypes: float64(1), object(1)
memory usage: 324.0+ bytes


yearmon      0
benchmark    0
dtype: int64

In [426]:

#normalize the data columns between the tables
soda_data['yearmon'] = '20' + soda_data['transaction_date'].str.split('/').str[-1] +'-' + soda_data['transaction_date'].str.split('/').str[0]
soda_data['yearmon'] = soda_data['yearmon'].apply(lambda x: str(x) if len(str(x)) == 7 else str(x)[:5] + '0' + str(x)[5:])
soda_data[['year', 'month']] = soda_data['yearmon'].str.split('-', expand=True)


In [427]:
#index user_id, check for missing values

soda_data.set_index('user_id')
soda_data.info()
soda_data.isnull().sum() 
# 30 missing values for total spent

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29798 entries, 0 to 29797
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             29798 non-null  int64  
 1   demographic_weight  29798 non-null  float64
 2   basket_id           29798 non-null  int64  
 3   channel             29798 non-null  object 
 4   total_spent         29768 non-null  float64
 5   total_units         29798 non-null  int64  
 6   trips               29798 non-null  int64  
 7   transaction_date    29798 non-null  object 
 8   yearmon             29798 non-null  object 
 9   year                29798 non-null  object 
 10  month               29798 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 2.5+ MB


user_id                0
demographic_weight     0
basket_id              0
channel                0
total_spent           30
total_units            0
trips                  0
transaction_date       0
yearmon                0
year                   0
month                  0
dtype: int64

In [ ]:
# imputation of missing values

impute_method = IterativeImputer(max_iter = 5, random_state = 127)
imputed_values = impute_method.fit_transform(soda_data.iloc[:,~soda_data.columns.isin(['user_id','basket_id', 'channel', 'transaction_date', 'yearmon'])])
imputed_df = pd.DataFrame(imputed_values)


In [ ]:
#readjust the names of the imputed columns

imputed_df.columns = ["demographic_weight","total_spent", "total_units","trips"	,"year","month"]

In [ ]:
#merge the imputed df with original df

soda_final = soda_data.merge(imputed_df, left_index = True, right_index = True)
soda_final = soda_final.iloc[:, ~soda_final.columns.isin(['demographic_weight_x','total_spent_x', 'total_units_x', 'trips_x','year_x', 'month_x' ])]
soda_final.columns = ['user_id', 'basket_id', 'channel', 'transaction_date', 'yearmon','demographic_weight', 'total_spent', 'total_units', 'trips','year', 'month']

In [ ]:
#create a aggregate table for the sum of the table spent of the ledgar, and the proportion of the total spend compated to the
#benchmark grouped monthly

sum_sales = soda_final.groupby(['yearmon'])['total_spent'].sum().sort_values() 
sum_sales = pd.DataFrame(sum_sales)

sum_sales = sum_sales.merge(benchmark_data, on = "yearmon").sort_values(by = "yearmon")
sum_sales['coverage_rate'] = sum_sales['total_spent']/sum_sales['benchmark'] * 100
sum_sales.columns = ["yearmon","agg_total_spent","benchmark","coverage_rate"]
sum_sales

In [ ]:
#create additional variables describing the portionof total sales attributed to each user and the estimate of the 
#proportion of the benchmark

full_data = soda_final.merge(sum_sales, on = "yearmon")
full_data['sales_proportion'] = full_data['total_spent']/full_data['agg_total_spent']
full_data['bench_estimate'] = full_data['sales_proportion'] * full_data['benchmark']

In [ ]:
from sklearn.preprocessing import StandardScaler

#create alias for transformations of the data used for modeling
model_data = full_data

In [ ]:
#subset out variables excluded from modeling or modeling transformations

model_data = model_data.iloc[:, ~model_data.columns.isin(["user_id", "basket_id", "channel", "transaction_date", "benchmark", "coverage_rate", "agg_total_spent", "sales_proportion"])]

In [ ]:

#transform date metrics into strings to furhter convert into ordinal variables

model_data['year'] = model_data['year'].apply(str)
model_data['month'] = model_data['month'].apply(str)


In [ ]:
# convert year and month variables into ordinal variables

ordinal_encoder_year = OrdinalEncoder(categories = [["2020.0", "2021.0"]])
ordinal_encoder_month = OrdinalEncoder(categories = [[ '1.0','2.0', '3.0', '4.0','5.0', '6.0','7.0','8.0', '9.0', '10.0','11.0','12.0']])

model_data['year'] = ordinal_encoder_year.fit_transform(model_data['year'].values.reshape(-1,1))
model_data['month'] = ordinal_encoder_month.fit_transform(model_data['month'].values.reshape(-1,1))


In [ ]:
#separate response and features into separate dataframes

feature_set = model_data.iloc[:,:-1]
response_var = model_data.iloc[:,-1]

In [ ]:
#create a manual training and test set based time based separations, to set up for future forecasting

x_train = model_data.iloc[:,:-1][model_data["yearmon"] < "2021-03"]
y_train = model_data.iloc[:,-1][model_data["yearmon"] < "2021-03"]
x_test = model_data.iloc[:,:-1][model_data["yearmon"] >= "2021-03"]
y_test = model_data.iloc[:,-1][model_data["yearmon"] >= "2021-03"]

x_train = x_train.iloc[:,1:]
x_test = x_test.iloc[:,1:]

In [ ]:
# create a small grid search function to find the most optimal hyperparameters for the gradient boosting model


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
def grid_search_models(x_train_df, y_train_df, num_estimators_per_param, random_state): 
        grid = {'n_estimators': [n for n in range(500,3000,500)[0: num_estimators_per_param]],
              'min_child_weight': [n for n in range(1,20,1)[0: num_estimators_per_param]],
              'max_depth': [n for n in range(1,10,1)[0: num_estimators_per_param]]}
        xgb_grid_search = GridSearchCV(XGBRegressor(random_state = random_state), param_grid = grid,
                          cv = KFold(n_splits = 5, shuffle = True, random_state = random_state), scoring = 'f1')
        xgb_grid_search.fit(x_train_df, y_train_df)
        return(xgb_grid_search.best_params_)
    

In [ ]:
#run the model using the parameters determined by the hyperparameter tuning function

xgb_params = grid_search_models(x_train, y_train, 3, 10)

regressor = XGBRegressor(n_estimators = xgb_params['n_estimators'], min_child_weight = xgb_params['min_child_weight'], max_depth = xgb_params['max_depth'], booster = "gbtree")
regressor.fit(x_train, y_train)
predictions = pd.DataFrame(regressor.predict(x_test))



In [ ]:
# rename the column for the prediction
predictions.columns = ["benchmark_prediction"]
predictions


In [ ]:
# reset the index to be able to merge the testing data with the original dataset
x_test = x_test.reset_index()
x_test


In [ ]:
#merge the results of the predictions with the full testing dataset
pred_merge = x_test.merge(predictions, left_index = True, right_index = True)
pred_merge


In [ ]:
#merge the predictions df with the original variables from the sales and benchmark dataframes
pred_merge = pred_merge.merge(full_data.iloc[:, full_data.columns.isin(['yearmon', 'benchmark', 'sales_proportion', 'coverage_rate'])], left_on = "index", right_index = True)


In [ ]:
#organize the benchmark data for forecasting and answering the follow up questions

benchmark_pred_df = pd.DataFrame(pred_merge.groupby(['yearmon'])['benchmark_prediction'].sum().sort_values())
benchmark_pred_df = benchmark_pred_df.merge(benchmark_data, left_index = True, right_on = "yearmon")
benchmark_pred_df['yearmon'] = pd.to_datetime(benchmark_pred_df['yearmon'])
benchmark_pred_df = benchmark_pred_df.set_index('yearmon')
benchmark_pred_df

2. What is the projected total$sales for the given year?

In [ ]:
# import packages to train a time series and forecast yearly sales

from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.tsa.arima_model import  ARIMA
from sklearn import model_selection



In [ ]:
# build the timeseries using standard orders and seasonality
# fit the model to the testing data results of the boosting model

ts_model = sm.tsa.statespace.SARIMAX(benchmark_pred_df['benchmark_prediction'],
                                order=(1, 1, 1),
                                seasonal_order=(0, 1, 1, 2),
                                enforce_stationarity=False,
                                enforce_invertibility=True)
ts_fit = ts_model.fit()
predictions = ts_fit.get_prediction(start=pd.to_datetime('2021-03-01'), dynamic = True)
print(ts_fit.summary().tables[1])


In [ ]:
# produce the confidence intervals for the timeseries

prediction_ci = predictions.conf_int()
ax = benchmark_pred_df.plot(label = "benchmark", figsize=(15, 7))
predictions.predicted_mean.plot(ax = ax,alpha = 1)
ax.fill_between(prediction_ci.index, 
                prediction_ci.iloc[:, 0], 
                prediction_ci.iloc[:, 1], 
                color = "k", alpha = 0.05)

In [ ]:
#rename the benchmark data in the format for the forecast, specific to prophet package

benchmark_pred_ts = benchmark_pred_df
benchmark_pred_ts.columns = ["ds","y", "observed"]
benchmark_pred_ts


In [ ]:
# install packages for forecasting a year point forward based on the results of the boosting model
from prophet import Prophet

forecast = Prophet()
forecast_pred = forecast.fit(benchmark_pred_ts)

forecast_results = forecast.make_future_dataframe(periods = 24, freq = "M")
fore_predictions = forecast.predict(forecast_results)

# print the forecasting values as well as upper and lower bounds in comparison
fore_predictions[["ds", "yhat", "yhat_lower", "yhat_upper"]]




3. What is the projected household penetration for the given year? Household penetration is defined as (# Households Involved in Purchase / Total US Households)


In [ ]:
# time box the forecast predictions for a year time period

spend_predict = pd.DataFrame(fore_predictions[["ds", "yhat"]])
spend_predict = spend_predict[(spend_predict["ds"] > "2021-08-01") & (spend_predict["ds"] < "2022-10-01")]


In [ ]:
# compute the yearly sum of sales based on the predictions

predicted_sum = int(spend_predict['yhat'].sum())
predicted_sum


In [ ]:
# approximate the averge proportion of sales in comparison to the benchmark using the individual user sales proportions
# in comparison to the total monthly sales

avg_prop = ((prop_table['sales_proportion']/prop_table['coverage_rate']) / prop_table['benchmark']).mean()
avg_prop
us_households = 151310000


In [ ]:
# compute the percent estimate of household penetration

household_penetration = ((predicted_sum * avg_prop) / us_households) * 100


In [ ]:
# using the metrics used to estimate household penetration, estimate how the individual receipt collection
# of the recorded sales is reflected as a proportion of the benchmark to understand the reciept penetration.

pred_merge.groupby(['yearmon'])['sales_proportion'].count().sort_values()

prop_table = sum_sales.merge(pred_merge.groupby(['yearmon'])['sales_proportion'].count().sort_values(), on = "yearmon")
prop_table

recepit_penetration = (prop_table['sales_proportion']/prop_table['coverage_rate']) / prop_table['benchmark'] * 100


